In [9]:
%run ../Exercises-1/rag_qna_bot.ipynb
%run ../Exercises-1/summarize_bot.ipynb

{'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2024-05-14T22:16:56+02:00', 'author': 'BMW AG', 'moddate': '2024-05-17T08:49:01+02:00', 'subject': 'Speech of the CEO Oliver Zipse (full text)', 'title': 'Speech of the CEO Oliver Zipse (full text)', 'source': '../input/Zipse_Speech.pdf', 'total_pages': 17, 'page': 1, 'page_label': '2'}
82
gpt-4o-mini
{'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2024-05-14T22:16:56+02:00', 'author': 'BMW AG', 'moddate': '2024-05-17T08:49:01+02:00', 'subject': 'Speech of the CEO Oliver Zipse (full text)', 'title': 'Speech of the CEO Oliver Zipse (full text)', 'source': '../input/Zipse_Speech.pdf', 'total_pages': 17, 'page': 1, 'page_label': '2'}
17


In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
from openai import AzureOpenAI
from dotenv import load_dotenv
import os,json

load_dotenv()

True

In [11]:
# Azure Keys
azure_endpoint: str = os.environ.get('AZURE_ENDPOINT')
azure_openai_api_key: str = os.environ.get('AZURE_OPENAI_API_KEY')
azure_openai_api_version: str = os.environ.get('AZURE_OPENAI_API_VERSION')
azure_deployment: str = os.environ.get('AZURE_DEPLOYMENT')

In [12]:
client = AzureOpenAI(
    api_key=azure_openai_api_key,
    api_version=azure_openai_api_version,
    azure_endpoint = azure_endpoint
)

In [24]:
def classify_intent(user_input):
    # Define the system prompt for intent classification
    system_prompt = """
    You are an intent classification system. Your task is to classify user inputs into one of two categories:
    1. Q&A: The user is asking a question and expects an answer.
    2. Summary: The user is asking for a summary of content.
    
    For each input, return a JSON object with:
    - "intent": Either "Q&A" or "Summary"
    - "confidence": A score between 0 and 1 indicating your confidence
    - "explanation": A brief explanation of why you classified it this way
    """
    
    # Create messages for the API call
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]
    
    # Call Azure OpenAI API
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        temperature=0.1,  # Low temperature for more deterministic responses
        response_format={"type": "json_object"}
    )
    
    # Parse the response
    result = json.loads(response.choices[0].message.content)
    return result

In [25]:
def check_content_policy(user_input):
    system_prompt = """
    You are a content policy checker. Your task is to determine if the user input contains sensitive topics 
    that should be avoided in responses. The topics to avoid are:
    
    - Religion: Religious discussions, debates about faith, religious practices, criticism of religions
    - Adult content: Explicit sexual content, pornography, sexual acts
    - Illegal activities: Instructions for illegal activities, promotion of criminal behavior
    - Weapons and violence: Detailed instructions about weapons, promotion of violence
    - Medical advice: Specific medical diagnoses, treatment recommendations, health advice
    - Financial investment advice: Specific investment recommendations, financial predictions
    - Personal identifiable information: Requests for or sharing of personal data
    
    Return with a JSON object containing:
    - "is_allowed": boolean (True if content is allowed, False if it contains sensitive topics)
    - "reason": string explaining why the content is not allowed (if applicable)
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]
    
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        temperature=0.1,
        response_format={"type": "json_object"}
    )
    
    result = json.loads(response.choices[0].message.content)
    return result

In [38]:

def chat_bot(question):
  response=classify_intent(question)
  print(f'Intent Identified By AI : {response.get('intent')}')
  if response.get('intent') == 'Q&A':
    response=ask_question(question)
    policy_json=check_content_policy(response)
    if policy_json.get('is_allowed'):
      return response
    else:
      return policy_json.get('reason')
  elif response.get('intent') == 'Summary':
    response=run_summary()
    policy_json=check_content_policy(response)
    if policy_json.get('is_allowed'):
      return response
    else:
      return policy_json.get('reason')
  else:
    return 'Could not Find Intent'

In [39]:
question = 'when the speech happned'
print(chat_bot(question))

Intent Identified By AI : Q&A
| Event                                    | Date              |
|------------------------------------------|-------------------|
| 104th Annual General Meeting of BMW AG  | 15th May 2024     |
